# mecab, pororo 설치 필요

1. mecab 설치
- 진명훈님 mecab 설치법 (https://stages.ai/competitions/75/discussion/talk/post/609)
2. python-mecab-ko 설치
- pip install python-mecab-ko
3. pororo 설치
- pip install pororo

# 데이터 convert

In [10]:
import json

def mrc_json_to_list(json_path):
    with open(json_path) as f:
        json_object = json.load(f)
    return list(json_object.values())

def mrc_csv_to_list(csv_path):
    df = pd.read_csv(csv_path)
    df["answers"] = df["answers"].fillna('[""]')
    return list(df["answers"].map(lambda x : eval(x)))



# SQUAD [em,f1] score 계산하기

In [3]:
from datasets import load_metric
import json
import pandas as pd

metric = load_metric("squad")

def get_squad_score(predict_list, reference_list):
    predictions = [{"prediction_text": data, "id": str(i)} for i, data in enumerate(predict_list)]
    references = [ {"answers": {"answer_start": [0], "text": data}, "id": str(i)} for i, data in enumerate(reference_list)]
    
    log = metric.compute(predictions=predictions, references=references)
    
    print(log)
    print(f"correct_num : {log['exact_match']* 600/100}/{len(predictions)}")


Using the latest cached version of the module from /opt/ml/.cache/huggingface/modules/datasets_modules/metrics/squad/513bf9facd7f12b0871a3d74c6999c866ce28196c9cdb151dcf934848655d77e (last modified on Sun Oct 31 12:51:29 2021) since it couldn't be found locally at squad, or remotely on the Hugging Face Hub.
Exception ignored in: <function tqdm.__del__ at 0x7f82415140d0>
Traceback (most recent call last):
  File "/opt/conda/envs/pororo/lib/python3.8/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/opt/conda/envs/pororo/lib/python3.8/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


# 마지막 조사 제거

In [19]:
from pororo import Pororo

pos = Pororo(task="pos", lang="ko")

#마지막이 조사일 경우에 제거해주기. ex) 감각기에 -> 감각기
def remove_joint_tag(predicts : list, references : list = None, show_removed_joint : bool = True):
    new_predicts = []
    
    for i, predict in enumerate(predicts):    
        # pos("안녕하세요") -> [('안녕', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('어요', 'EF')]
        tag_list = pos(predict)
        last_tag = tag_list[-1][1]

        # JKS 주격 조사  JKC 보격 조사  JKG 관형격 조사  JKO 목적격 조사  JKB 부사격 조사  JKV 호격 조사  JKQ 인용격 조사  JX 보조사  JC 접속 조사
        if "J" in last_tag:
            start_index = predict.rfind(last_tag)
            new_predicts.append(predict[:start_index])

            if show_removed_joint and (references != None):
                print(f"{i} || pred : {tag_list}|| removed_pred : {predict[:start_index]} || ref : {references[i]} ")
        else:
            new_predicts.append(predict)
    return new_predicts

raw_predict_list = mrc_json_to_list("/opt/ml/723.json")
reference_list = mrc_csv_to_list("/opt/ml/sudo_test.csv")
predict_list_no_joint = remove_joint_tag(raw_predict_list, reference_list)

get_squad_score(predict_list_no_joint, reference_list)


15 || pred : [('그', 'MM'), ('로인', 'NNG'), ('볼트', 'NNG'), ('로', 'JKB')]|| removed_pred : 그로인볼트 || ref : ['석조 궁륭'] 
59 || pred : [('남미', 'NNP'), (' ', 'SPACE'), ('자브', 'NNP'), ('로', 'JKB')]|| removed_pred : 남미 자브 || ref : ['남미 자브로', '자브로'] 
166 || pred : [('고려', 'NNG'), (' ', 'SPACE'), ('하대', 'NNG'), ('로', 'JKB')]|| removed_pred : 고려 하대 || ref : ['17세기'] 
177 || pred : [('아루', 'NNP'), ('에', 'JKB')]|| removed_pred : 아루 || ref : ['아루에'] 
183 || pred : [('미술', 'NNG'), ('과', 'JC')]|| removed_pred : 미술 || ref : ['건축학과'] 
253 || pred : [('감각', 'NNG'), ('기', 'NNG'), ('에', 'JKB')]|| removed_pred : 감각기 || ref : ['감각기'] 
340 || pred : [('법무성', 'NNP'), ('이', 'JKS')]|| removed_pred : 법무성 || ref : ['법무성'] 
355 || pred : [('히스토리아', 'NNP'), ('가', 'JKS')]|| removed_pred : 히스토리아 || ref : ['미카사 아커만', '미카사'] 
480 || pred : [('베다', 'NNG'), ('는', 'JX')]|| removed_pred : 베다 || ref : ['베다'] 
{'exact_match': 73.33333333333333, 'f1': 76.50003607503608}
correct_num : 440.0/600


# 소괄호() 제거

In [5]:
import re

def remove_bracket(predict_list):
    temp = []
    for i, predict in enumerate(predict_list):
        re_result = re.compile(".*\(.*\)$").match(predict)
        if (re_result) != None:
            # print(predict)
            temp.append(re.compile("\(.*\)$").sub("",predict))
        else:
            temp.append(predict)
    return temp

# 따옴표 제거

In [6]:
import re

def remove_quotes(predict_list):
    temp = []
    for i, predict in enumerate(predict_list):
        
        # 역의 경우
        # if re.compile("^'\w*'$").match(ref) != None:
        #     print(i, str_tag, ref)
        
        if (re.compile("^'[\w\s]*'$").match(predict) != None) or re.compile('^"[\w\s]*"$').match(predict) != None:
            # if predict not in ref:
            #     print(i, predict, ref)
            temp.append(eval(predict))
        else:
            temp.append(predict)
    return temp


# post-processing  [ 조사제거 -> 소괄호제거 ]

In [12]:
raw_predict_list = mrc_json_to_list("/opt/ml/723.json")

reference_list = mrc_csv_to_list("/opt/ml/sudo_test.csv")

print("original score")
get_squad_score(raw_predict_list, reference_list)

predict_list_no_joint = remove_joint_tag(raw_predict_list)
print("조사 제거 score")
get_squad_score(predict_list_no_joint, reference_list)

predict_list_no_bracket = remove_bracket(raw_predict_list)
print("소괄호 제거 score")
get_squad_score(predict_list_no_bracket, reference_list)

original score
{'exact_match': 73.16666666666667, 'f1': 76.25003607503608}
correct_num : 439.0/600
조사 제거 score
{'exact_match': 73.33333333333333, 'f1': 76.50003607503608}
correct_num : 440.0/600
소괄호 제거 score
{'exact_match': 73.33333333333333, 'f1': 76.38892496392496}
correct_num : 440.0/600


# json 파일로 저장

In [13]:
def save_json(predict_list, id_list, save_name):
    all_predictions = {}

    for id, pred in zip(id_list, predict_list):
        all_predictions[id] = pred

    with open(save_name, "w", encoding="utf-8") as writer:
        writer.write(
            json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n"
        )

# 틀린 정답 print

In [15]:
def wrong_print(predict_list,reference_list):
    wrong_num = 0
    for j, (pred, ref) in enumerate(zip(predict_list, reference_list)):    
        if not pred in ref:
            print(f"{j} pred {pred} || refs : {ref}")
    print(wrong_num)

# 일부 포함 print

In [16]:
def sub_print(predict_list,reference_list):
    wrong_num = 0
    for j, (pred, ref) in enumerate(zip(predict_list, reference_list)):
        
        if not pred in ref:
            for refs in ref:
                if refs in pred:
                    wrong_num += 1
                    print(f"{j} pred {pred} || refs : {refs}")
    print(wrong_num)
